In [0]:
from google.colab import files
import io
import numpy as np  
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

#sklearn preparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#sklearn metrics
from sklearn.metrics import r2_score, mean_squared_error

#sklearn models
from sklearn import svm, linear_model, tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KDTree

In [0]:
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
diamonds = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')))

#Test dataset

In [0]:
test_up = files.upload()

KeyboardInterrupt: ignored

In [0]:
diamonds = pd.read_csv(io.StringIO(test_up['test.csv'].decode('utf-8')))

# Views

In [0]:
diamonds.shape

(13485, 10)

In [0]:
diamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,Ideal,G,SI2,61.9,57.0,8.44,8.36,5.20,12831
1,2.43,Very Good,H,SI2,63.2,57.0,8.56,8.50,5.39,16170
2,0.80,Premium,F,SI2,61.0,57.0,6.03,6.01,3.67,2797
3,0.40,Ideal,F,I1,63.3,60.0,4.68,4.64,2.95,630
4,0.31,Ideal,G,VS2,61.6,55.0,4.39,4.37,2.70,698


In [0]:
diamonds.describe()

,id,carat,depth,table,x,y,z
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,6742.000000,0.796605,61.753823,57.451479,5.728506,5.730286,3.536324
std,3892.928525,0.473000,1.433563,2.244549,1.117562,1.109549,0.693549
min,0.000000,0.200000,53.100000,49.000000,0.000000,0.000000,0.000000
25%,3371.000000,0.400000,61.100000,56.000000,4.710000,4.720000,2.910000
50%,6742.000000,0.700000,61.800000,57.000000,5.690000,5.710000,3.520000
75%,10113.000000,1.040000,62.500000,59.000000,6.540000,6.540000,4.030000
max,13484.000000,4.130000,79.000000,95.000000,10.140000,10.100000,6.430000


In [0]:
diamonds.dtypes

id           int64
carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
x          float64
y          float64
z          float64
dtype: object

### Correlations

In [0]:
diamonds[["carat","x","y","z"]].corr()

,carat,x,y,z
carat,1.000000,0.975010,0.944669,0.948835
x,0.975010,1.000000,0.967161,0.966188
y,0.944669,0.967161,1.000000,0.941816
z,0.948835,0.966188,0.941816,1.000000


The four columns carat, x, y, z are strictly correlated. I keep the one with highest correlation with the rest (x), and remove the rest.


In [0]:
diamonds.drop(["carat","y","z"], axis=1, inplace=True)

In [0]:
diamonds.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.56,Ideal,I,VS2,62.1,54.0,5.30,5.33,3.30
1,1,1.05,Ideal,G,VS2,61.9,56.0,6.56,6.52,4.05
2,2,0.50,Premium,E,VS2,61.5,56.0,5.11,5.07,3.13
3,3,0.80,Ideal,F,VS1,62.8,56.0,5.91,5.96,3.73
4,4,0.54,Ideal,G,VS1,61.4,57.0,5.23,5.20,3.20


# Scaled


In [0]:
scalar_features = diamonds[["depth", "table", "x"]]
gt = diamonds["price"]
scaler = StandardScaler()
scaler.fit(scalar_features, gt)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
transformed_scalar_features = scaler.transform(scalar_features)
scaled = pd.DataFrame(transformed_scalar_features, columns=["depth", "table", "x"])

In [0]:
scaled.head()

,depth,table,x
0,0.106170,-0.205764,2.411028
1,1.013798,-0.205764,2.517870
2,-0.522188,-0.205764,0.265287
3,1.083616,1.138849,-0.936684
4,-0.103283,-1.102173,-1.194885


#Object Columns (Replace categories for numbers):

In [0]:
categorical = diamonds[["cut", "color", "clarity"]]
categorical.head()

,cut,color,clarity
0,Ideal,G,SI2
1,Very Good,H,SI2
2,Premium,F,SI2
3,Ideal,F,I1
4,Ideal,G,VS2


In [0]:
categorical.replace(['Fair', 'Good', 'Ideal', 'Premium', 'Very Good'],[-2,-1, 0, 1, 2], inplace=True)
categorical.replace(['I1', 'SI2', 'SI1', 'VS2','VS1', 'VVS2', 'VVS1','IF'],[-4, -3, -2, -1, 0, 1, 2, 3], inplace=True)
categorical.replace(['D', 'E', 'F', 'G', 'H', 'I', 'J'],[-3, -2,-1, 0, 1, 2, 3], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [0]:
categorical

,cut,color,clarity
0,0,0,-3
1,2,1,-3
2,1,-1,-3
3,0,-1,-4
4,0,0,-1
...,...,...,...
40450,1,1,-2
40451,0,-1,-1
40452,2,2,0
40453,0,0,-3


### Object columns (Get Dummies):

In [0]:
set(diamonds['cut'])

{'Fair', 'Good', 'Ideal', 'Premium', 'Very Good'}

In [0]:
set(diamonds['color'])

{'D', 'E', 'F', 'G', 'H', 'I', 'J'}

In [0]:
set(diamonds['clarity'])

{'I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2'}

In [0]:
dummies = pd.get_dummies(diamonds[["cut", "color", "clarity"]])

In [0]:
dummies.head()

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


# Merge scaled with dummies or categorical

In [0]:
diamonds_fixed = scaled.merge(categorical, left_index=True, right_index=True)

In [0]:
diamonds_fixed.head()

,depth,table,x,cut,color,clarity
0,0.106170,-0.205764,2.411028,0,0,-3
1,1.013798,-0.205764,2.517870,2,1,-3
2,-0.522188,-0.205764,0.265287,1,-1,-3
3,1.083616,1.138849,-0.936684,0,-1,-4
4,-0.103283,-1.102173,-1.194885,0,0,-1


In [0]:
diamonds_fixed.corr()[diamonds.corr() >= 0.9]

,depth,table,x,cut,color,clarity
depth,1.0,NaN,NaN,NaN,NaN,NaN
table,NaN,1.0,NaN,NaN,NaN,NaN
x,NaN,NaN,1.0,NaN,NaN,NaN
cut,NaN,NaN,NaN,NaN,NaN,NaN
color,NaN,NaN,NaN,NaN,NaN,NaN
clarity,NaN,NaN,NaN,NaN,NaN,NaN


#X & y

In [0]:
X = diamonds_fixed[list(set(diamonds_fixed.columns))]

In [0]:
y = diamonds['price']

In [0]:
print(X.shape, y.shape)

(40455, 6) (40455,)


# Model Preparation


In [0]:
#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [0]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(36409, 6) (4046, 6) (36409,) (4046,)


# Trying Models:

In [0]:
models = {
    "RandomForest": RandomForestRegressor(),
    "SGDRegressor": linear_model.SGDRegressor(max_iter=1000, tol=1e-3),
    "DecisionTree" : tree.DecisionTreeRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "AdaBoostRegressor" : AdaBoostRegressor()    
}
#"KDTree": KDTree(X_train, leaf_size=30, metric='euclidean'),
for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)

d = {modelName:model.predict(X_test) for modelName, model in models.items()}
df = pd.DataFrame(d)
df["gt"] = y_test.reset_index(drop=True)

switchColor = lambda value: f"<b style=\"color:orange\">{round(value,2)}</b>" if value < 0.8 else f"<b style=\"color:green\">{round(value,2)}</b>"
printBonito = lambda label,val: display(HTML(f"<span style=\"padding-left:20px\">The {label} is: {switchColor(val)}</span>"))

for modelName, model in models.items():    
    print(f"Evaluating model [{modelName}]:")
    printBonito("r2_score", r2_score(df["gt"],df[modelName]))
    printBonito("RSME", (mean_squared_error(df["gt"],df[modelName])**0.5))

Training model: RandomForest


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training model: SGDRegressor
Training model: DecisionTree
Training model: GradientBoostingRegressor
Training model: AdaBoostRegressor
Evaluating model [RandomForest]:


Evaluating model [SGDRegressor]:


Evaluating model [DecisionTree]:


Evaluating model [GradientBoostingRegressor]:


Evaluating model [AdaBoostRegressor]:


# Gradient Boosting Regressor:


In [0]:
### Trying Parameters:
loss = ["huber"]
n_estimators = [2000]
# min_samples_leaf = [10, 50, 100]
# min_samples_split = [10, 50, 100]
for n in n_estimators:
  # for l in min_samples_leaf:
  #   for s in min_samples_split:
      gdr = GradientBoostingRegressor(n_estimators=n, loss="huber")
      print(f"number of trees: {n}")
      gdr.fit(X_train, y_train)
      prediction = gdr.predict(X_test)
      print("r2_score", r2_score(y_test,prediction))
      print(f"RSME {(mean_squared_error(y_test,prediction))**0.5} \n")

number of trees: 2000
r2_score 0.9801362863476932
RSME 556.4299990658022 



In [0]:
chosen_gbr = GradientBoostingRegressor(n_estimators=3000, loss="huber") 
chosen_gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='huber', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
gbr_prediction = chosen_gbr.predict(X_test)
print("r2_score", r2_score(y_test,gbr_prediction))
print(f"RSME {(mean_squared_error(y_test,gbr_prediction))**0.5} \n")

r2_score 0.9796631847099331
RSME 578.59675398023 



In [0]:
chosen_gbr_prediction = chosen_gbr.predict(X)

# Random Forest

In [0]:
#RandomForestRegressor

### Trying Parameters:
n_estimators = [900, 1000, 1100]
min_samples_split = range(10, 100, 20)
min_impurity_decrease = range(10, 100, 20)
for n in n_estimators:
#  for s in min_samples_split:
      forest = RandomForestRegressor(n_estimators=n, n_jobs=-1, warm_start=True)
      print(f"number of trees: {n}")
      forest.fit(X_train, y_train)
      prediction = forest.predict(X_test)
      print("r2_score", r2_score(y_test,prediction))
      print("RSME", mean_squared_error(y_test,prediction))

number of trees: 100
r2_score 0.9780147359014683
RSME 348799.7528610012
number of trees: 400
r2_score 0.9782098687072103
RSME 345703.93949651934
number of trees: 700
r2_score 0.9781987143658377
RSME 345880.90491738706
number of trees: 1000
r2_score 0.9782570037484324
RSME 344956.13448241295
number of trees: 1300
r2_score 0.9781653298396072
RSME 346410.5557983764
number of trees: 1600
r2_score 0.9782343671401134
RSME 345315.26791614864
number of trees: 1900
r2_score 0.9782431295704628
RSME 345176.2505485762


In [0]:
#Fitting RandomForestRegressor
chosen_forest = RandomForestRegressor(1500, n_jobs=-1)
chosen_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1500,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [0]:
forest_prediction = chosen_forest.predict(X_test)
print(r2_score(y_test, forest_prediction))
print((mean_squared_error(y_test, forest_prediction))**0.5)

0.9767964977191061
625.2194459441428


In [0]:
chosen_gbr_prediction

array([ 1265.68466839,  6650.2362962 ,  1679.18867163, ...,
       11466.16533155,   460.21784355,  1348.37381879])

In [0]:
test_submit = pd.DataFrame({
    "price" : chosen_gbr_prediction
}, index=diamonds['id'])

In [0]:
test_submit

,price
id,
0,1265.684668
1,6650.236296
2,1679.188672
3,3670.470329
4,1793.574763
...,...
13480,2149.838497
13481,1974.276142
13482,11466.165332


In [0]:
test_submit.to_csv("diamonds_submit_1.csv")
files.download("diamonds_submit_1.csv")